In [1]:
import pandas as pd
from pathlib import Path  
import sys
import sqlite3

In [23]:
data_filtered=pd.read_csv("data_filtered.csv")


data_filtered['M0906_DC_TRAN_DTH_DT'] = pd.to_datetime(data_filtered['M0906_DC_TRAN_DTH_DT'], infer_datetime_format=True)
data_filtered['cedd_dateofdeath'] = pd.to_datetime(data_filtered['cedd_dateofdeath'], infer_datetime_format=True)
data_filtered['epi_SocDate'] = pd.to_datetime(data_filtered['epi_SocDate'], infer_datetime_format=True)
data_filtered[["DaysToDeath"]] = data_filtered[["DaysToDeath"]].apply(pd.to_numeric)


data_max_dc_date=data_filtered[(data_filtered['dcr_desc']=='PATIENT EXPIRED') & (data_filtered['DaysToDeath'].isnull())].groupby(by=['pa_id'])[['M0906_DC_TRAN_DTH_DT']].max()
data_max_dc_date=data_max_dc_date.add_suffix('_Date').reset_index()
def populate_date_of_death(x):
    if((x['dcr_desc']=='PATIENT EXPIRED') & (pd.isnull(x['cedd_dateofdeath']))):
        return data_max_dc_date[data_max_dc_date['pa_id']==x['pa_id']]['M0906_DC_TRAN_DTH_DT_Date'].values[0]
    return x['cedd_dateofdeath']
    

data_filtered['cedd_dateofdeath']=data_filtered.apply(populate_date_of_death, axis=1)
data_filtered['DaysToDeath'] = (data_filtered['cedd_dateofdeath'] - data_filtered['epi_SocDate']).dt.days

filepath = Path('data_filtered_with_cedd.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
data_filtered.to_csv(filepath)  



In [25]:
def getColumnDtypes(dataTypes):
    dataList = []
    for x in dataTypes:
        if(x == 'int64'):
            dataList.append('int')
        elif (x == 'float64'):
            dataList.append('float')
        elif (x == 'bool'):
            dataList.append('boolean')
        else:
            dataList.append('varchar')
    return dataList
data_filtered = data_filtered.drop('Unnamed: 0', 1)
# display(data_filtered)
columnName = list(data_filtered.columns.values)
# display(columnName[-1:])
columnDataType = getColumnDtypes(data_filtered.dtypes)
clearTable='DROP TABLE IF EXISTS data_updated;'
createTableStatement = 'CREATE TABLE IF NOT EXISTS data_updated ('
for i in range(len(columnDataType)):
    createTableStatement = createTableStatement + '\n' + columnName[i] + ' ' + columnDataType[i] + ','
createTableStatement = createTableStatement[:-1] + ' );'
Path('compassus_data_updated.db').touch()
conn = sqlite3.connect('compassus_data_updated.db')
c = conn.cursor()
# display(createTableStatement)
c.execute(clearTable)
c.execute(createTableStatement)

data_filtered.to_sql('data_updated', conn, if_exists='append', index = False)